In [30]:
import os, time
import requests
from selenium import webdriver
import pandas as pd
import urllib.request
import datetime
from selenium.webdriver.common.keys import Keys

In [9]:
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.implicitly_wait(3)
        if datetime.datetime.now() > end:
            break

In [10]:
# 드라이버 지정
current = os.getcwd()
chrome_path = current + '/driver/chromedriver' 

options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
options.add_argument('headless') # 창 안 띄움 -> 더 빠름
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # Headless 탐지 막기위해 User-Agent 값을 바꿔줌.

options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

In [32]:
columns = ["id", "title", "type", "open_year", "rate", "running_time", "genre", "actor", "director", "summary"]
movie_datas = []

driver = webdriver.Chrome(chrome_path, options=options)

# 영화
# url = "https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie"
# stream_type = "Movie"

# TV
url = "https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=show"   
stream_type = "TV Show"

driver.get(url)

num = driver.find_element_by_css_selector(".total-titles").text
num = int(num.split(' ')[0].replace(",", ""))

# doScrollDown(60)

In [6]:
from tqdm import tqdm

In [27]:
imgUrl = driver.find_elements_by_css_selector(".picture-comp__img")[0].get_attribute("src")

element = driver.find_elements_by_css_selector(".title-poster")[0]
driver.execute_script("arguments[0].click();", element)

driver.implicitly_wait(3)
title = driver.find_elements_by_css_selector('.title-block h1')[0].text
title


'그 해 우리는'

In [16]:
title = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]/div/h1').text

In [19]:
movie_data

[]

In [33]:
for i in tqdm(range(num)):
    try:
        imgUrl = driver.find_elements_by_css_selector(".picture-comp__img")[i].get_attribute("src")
        
        element = driver.find_elements_by_css_selector(".title-poster")[i]
        driver.execute_script("arguments[0].click();", element)
        time.sleep(2)
        
        movie_data = []
        actors = []
        
        id = i
        
        title = driver.find_elements_by_css_selector('.title-block h1')[0].text
            
        open_year = driver.find_element_by_css_selector(".text-muted").text
        open_year = open_year.strip().replace("(","").replace(")","")
        
        rate = driver.find_element_by_css_selector(".jw-scoring-listing__rating").text
        rate = int(rate.replace("%","")) / 10
        
        running_time = driver.find_elements_by_css_selector(".detail-infos__value")[2].text
        
        genre = driver.find_elements_by_css_selector(".detail-infos__value")[1].text
        genres = genre.replace(" ","").split(",")

        for j in range(2, 8):
            actor = driver.find_elements_by_css_selector(".title-credit-name")[j].text
            actors.append(actor)

        director = driver.find_elements_by_css_selector(".title-credit-name")[0].text
        try:
            summary = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[6]/div[1]/div[4]/p/span').text
        except:
            summary = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[5]/div[1]/div[4]/p/span').text

        movie_data.append(id)
        movie_data.append(title)
        movie_data.append(stream_type)
        movie_data.append(open_year)
        movie_data.append(rate)
        movie_data.append(running_time)
        movie_data.append(genres)
        movie_data.append(actors)
        movie_data.append(director)
        movie_data.append(summary)
        
        movie_datas.append(movie_data)
        
        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(imgUrl, "./posters/" + title + ".jpg")
        
        df = pd.DataFrame(movie_datas, columns=columns)
        df.to_csv('movies_data.csv', encoding='utf-8')
        
        driver.back()
    
    except:
        driver.back()
driver.close()

  2%|▏         | 33/1821 [01:27<1:18:37,  2.64s/it]


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54657): Max retries exceeded with url: /session/02da70b0d0076f26075472cb0341f688/back (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbd28ef35b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [34]:
df = pd.DataFrame(movie_datas, columns=columns)
df

,id,title,type,open_year,rate,running_time,genre,actor,director,summary
0,0,그 해 우리는,TV Show,2021,10.0,1시간 6분,"[드라마, 코미디]","[Choi Woo-shik, Kim Da-mi, Kim Sung-cheol, Roh...",Kim Yoon Jin,헤어진 연인이 고등학교 시절 촬영한 다큐멘터리의 역주행 인기로 강제 소환되면서 펼쳐...
1,5,일본 침몰 -희망의 사람-,TV Show,2021,8.0,53min,"[SF, 드라마]","[, , Shun Oguri, Kenichi Matsuyama, Anne Watan...",Nobuhiro Doi,"2023년, 태평양 해수면의 이례적인 급상승이 일본 동부를 위협한다. 부패한 관료,..."
2,11,엘프들,TV Show,2021,7.3,24min,"[가족, 드라마, 판타지, 공포, SF]","[Milo Campanale, Sonja Steen, Lila Nobel, Vive...",Roni Ezra,아름다운 섬에서 보내는 행복한 크리스마스를 상상했다. 그러나 이곳엔 피에 목마른 신...
3,14,모던 패밀리,TV Show,2009,8.9,21min,"[코미디, 드라마, 로맨스]","[Julie Bowen, Ty Burrell, Jesse Tyler Ferguson...",Ed O'Neill,에미상을 받은 이 시트콤은 로스앤젤레스에 사는 제이 프리쳇과 각양각색의 가족이 일상...
4,16,집필 불가! 각본가 케이스케 씨의 각본 없는 인생,TV Show,2021,7.9,30min,"[코미디, 드라마]","[Anna Yamada, Fuma Kikuchi, Michiko Kichise, T...",Masaki Okada,데뷔 5년 차 대타 전문 각본가 케이스케 씨에게 대박 찬스가 왔다. 황금 시간대 드...
5,17,La casa de papel - Netflix series,TV Show,2017,8.6,50min,"[드라마, 스릴러, 액션, 범죄, MadeinEurope]","[Itziar Ituño, Pedro Alonso, Jaime Lorente, Mi...",Álvaro Morte,An unusual group of robbers attempt to carry o...
6,19,DC 타이탄,TV Show,2018,8.8,45min,"[SF, 액션, 범죄, 드라마, 판타지]","[Anna Diop, Ryan Potter, Curran Walters, Conor...",Brenton Thwaites,복수에는 대가가 따른다. 정체를 숨기고 형사가 된 딕 그레이슨. 이제 그는 고담의 ...
7,21,Aranyak,TV Show,2021,8.8,41min,"[드라마, 범죄, 스릴러]","[Raveena Tandon, Ashutosh Rana, Parambrata Cha...",Vinay Waikul,"Political ploys, personal agendas and a beastl..."
8,25,아스코마치: 아스카 공업 고교 이야기,TV Show,2011,8.6,45min,[드라마],"[Renpei Tsukamoto, , , , Emi Takei, Tori Matsu...",Hibino Akira,1지망이었던 여고 입학 실패는 참담한 결과를 가져왔다. 남학생이 득실대는 공고 입학...
9,30,카케구루이,TV Show,2018,7.9,31min,"[드라마, 스릴러]","[Aoi Morikawa, Taishi Nakagawa, Yurika Nakamur...",Minami Hamabe,정재계 인사들의 자녀가 다수 다니는 사립 핫카오 학원에 주인공이 전학을 오면서 벌어...


In [3]:
df

,id,title,type,open_year,rate,running_time,genre,actor,director,summary
0,0,모가디슈,Movie,2021,7.9,2시간 1분,"[드라마, 액션, 스릴러]","[Kim Yoon-seok, Jo In-sung, Heo Joon-ho, Kim S...",Ryoo Seung-wan,유엔 가입을 위해 다수의 투표권을 지닌 아프리카 대륙에서 열심히 외교 활동을 벌이던...
1,1,베놈,Movie,2018,7.8,1시간 52분,"[액션, SF, 스릴러]","[Tom Hardy, Michelle Williams, Riz Ahmed, Scot...",Ruben Fleischer,진실을 위해서라면 몸을 사리지 않고 사회의 부조리를 취재하는 정의로운 열혈 기자 에...
2,2,스파이더맨,Movie,2002,8.4,2시간 1분,"[판타지, 로맨스, 액션, 드라마, SF]","[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...",Sam Raimi,평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후...
3,3,스파이더맨: 뉴 유니버스,Movie,2018,9.5,1시간 57분,"[코미디, 가족, 판타지, SF, 애니메이션, 액션]","[Bob Persichetti, , , , Shameik Moore, Jake Jo...",Rodney Rothman,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
4,4,스파이더맨: 홈커밍,Movie,2017,8.6,2시간 13분,"[SF, 액션]","[Tom Holland, Michael Keaton, Robert Downey Jr...",Jon Watts,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...
5,5,파워 오브 도그,Movie,2021,7.7,2시간 6분,"[드라마, 로맨스, 서부]","[Benedict Cumberbatch, Kirsten Dunst, Jesse Pl...",Jane Campion,사람들을 억누르면서도 사람들을 끌어당기는 묘한 매력의 목장주. 그의 동생이 새로운 ...
6,6,돈 룩 업,Movie,2021,8.3,2시간 23분,"[SF, 코미디, 드라마]","[Leonardo DiCaprio, Jennifer Lawrence, Meryl S...",Adam McKay,천문학과 대학원생 케이트 디비아스키(제니퍼 로렌스)와 담당 교수 랜들 민디 박사(레...
7,7,스파이더맨: 파 프롬 홈,Movie,2019,8.6,2시간 9분,"[액션, SF, 코미디]","[Tom Holland, Samuel L. Jackson, Jake Gyllenha...",Jon Watts,"어벤져스 멤버들과 타노스와의 대접전을 마친 뒤, 지난 5년 동안 사라졌던 사람들이 ..."
8,8,어메이징 스파이더맨,Movie,2012,7.9,2시간 11분,"[판타지, 액션, SF]","[Andrew Garfield, Emma Stone, Rhys Ifans, Deni...",Marc Webb,어릴적 사라진 부모 대신 삼촌 내외와 살고 있는 피터 파커는 평범한 학교 생활을 하...
9,9,더 플랫폼,Movie,2019,7.8,1시간 34분,"[SF, 스릴러, 공포, MadeinEurope]","[Iván Massagué, Antonia San Juan, Zorion Eguil...",Galder Gaztelu-Urrutia,30일마다 랜덤으로 레벨이 바뀌는 극한 생존의 수직 감옥 플랫폼. 최상위 레벨 0에...
